### __ИТОГОВЫЙ ПРОЕКТ__

#### __Обучение пайплайна__

In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline

from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

Загрузим данные

In [2]:
df = pd.read_csv("./train1.csv")
df.head(10)

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ
5,34554026,Accessing pointer member of the structure usin...,<p>I have defined integer pointer inside the s...,<c><pointers><data-structures>,2016-01-01 06:21:17,LQ_CLOSE
6,34554206,"How To Disable 2nd Saturday 4th Saturday ,Sund...","i want to Disable 2nd Saturday 4th saturday,...",<javascript><jquery><jquery-ui>,2016-01-01 06:55:18,LQ_EDIT
7,34554671,Resizing containers in bootstrap,I am new to bootstrap and i would like to find...,<jquery><html><css><twitter-bootstrap>,2016-01-01 08:32:05,LQ_EDIT
8,34554871,Changing Theme in Windows 10 UWP App Programma...,<p>I was able to change theme using <code>this...,<c#><windows-10><windows-10-mobile><windows-10...,2016-01-01 09:05:45,HQ
9,34554982,TextBox Value Disappears VB6,I my case I am having two form Form 1 and Form...,<vb6>,2016-01-01 09:27:44,LQ_EDIT


In [3]:
df['Y'].value_counts()

LQ_CLOSE    15000
HQ          15000
LQ_EDIT     15000
Name: Y, dtype: int64

Разделим данные на train/test и сохраним тестовую выборку на диск

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Y'],
                                                    test_size=0.3, random_state=42)
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [5]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    

class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [6]:
features = ['Title', 'Body', 'Tags']
target = 'Y'

Соберем кусок, ответственный за feature engineering

In [7]:
# combine
Title = Pipeline([
                ('imputer', TextImputer('Title', '')),
                ('selector', ColumnSelector(key='Title')),
                ('tfidf', TfidfVectorizer())
            ])

Body = Pipeline([
                ('imputer', TextImputer('Body', '')),
                ('selector', ColumnSelector(key='Body')),
                ('tfidf', TfidfVectorizer())
            ])

Tags = Pipeline([
                ('imputer', TextImputer('Tags', '')),
                ('selector', ColumnSelector(key='Tags')),
                ('tfidf', TfidfVectorizer())
            ])

feats = FeatureUnion([('Title', Title),
                      ('Body', Body),
                      ('Tags', Tags)])

Добавим простейший классификатор

In [8]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

In [9]:
pipeline.fit(X_train, y_train)

C:\Users\DS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Title',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Title',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='Title')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('Body',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Body',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='Body')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('Tags',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Tags',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='Tags')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('classifier', LogisticRegression())])

In [10]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('Title',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='Title',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='Title')),
                                                  ('tfidf', TfidfVectorizer())])),
                                 ('Body',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='Body',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='Body')),
                                                  ('tfidf', TfidfVectorizer())])),
                       

In [11]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

#### __Проверка работоспособности и качества пайплайна__

In [12]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [13]:
X_test.head(5)

,Id,Title,Body,Tags,CreationDate,Y
0,54233203,not able to run function in php,<p>I'am not able to run a function in php oops...,<php><oop>,2019-01-17 09:54:10,LQ_CLOSE
1,39253307,Export GIT LOG into an Excel file,"<p>I have looked into the forum, but with no l...",<git><logging>,2016-08-31 15:13:57,HQ
2,58793066,How to summarize the employees by net revenue ...,8.\tThe sales director would like to reward th...,<sql>,2019-11-10 21:25:11,LQ_EDIT
3,40792221,how to retrieve words based on user inputting ...,**how to retrieve words based on user inputtin...,<java><collections><treemap><treeset>,2016-11-24 17:45:18,LQ_EDIT
4,60256467,how to use val insted of var in scala,I simply want to remove var and use Val instea...,<java><scala><if-statement><collections><funct...,2020-02-17 05:38:36,LQ_EDIT


In [14]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [15]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Title',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Title',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='Title')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('Body',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Body',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='Body')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('Tags',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Tags',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='Tags')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('classifier', LogisticRegression())])

In [16]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [17]:
preds[:10]

array([0.97092715, 0.26582769, 0.06743919, 0.49850009, 0.10135996,
       0.4812224 , 0.18802748, 0.24348002, 0.39306495, 0.93939444])

### __Flask__

In [18]:
#!pip3 install flask-ngrok

In [19]:
from flask import Flask, request, jsonify
import pandas as pd

In [20]:
# Пробный запуск Flask

app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run(port=8082)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8082
Press CTRL+C to quit


In [21]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [22]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    description, company_profile, benefits = "", "", ""
    
    request_json = request.get_json()
    
    if request_json["description"]:
        description = request_json['description']
    
    if request_json["company_profile"]:
        company_profile = request_json['company_profile']
                
    if request_json["benefits"]:
        benefits = request_json['benefits']
    
    print(description)  
    preds = model.predict_proba(pd.DataFrame({"description": [description],
                                              "company_profile": [company_profile],
                                              "benefits": [benefits]}))
    data["predictions"] = preds[:, 1][0]
    data["description"] = description
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run(port=8082)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8082
Press CTRL+C to quit
